In [1]:
import numpy as np
import pandas as pd
import config # a python file that contains path to TIMSS data files

In [80]:
# Read both files in a dataframe
df_original_G8 = pd.read_spss(config.file_path_G8)
df_original_G4 = pd.read_spss(config.file_path_G4)

In [81]:
df_original_G8.shape

(8458, 460)

In [87]:
df_original_G4.shape

(7723, 244)

In [82]:
df_original_G8.dtypes

IDCNTRY     category
IDBOOK      category
IDSCHOOL    category
IDCLASS     category
IDSTUD      category
              ...   
BSDGEDUP    category
BSDMLOWP    category
BSDSLOWP    category
VERSION     category
SCOPE       category
Length: 460, dtype: object

In [83]:
df_original_G8.describe()

,IDCNTRY,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,BSBG01,BSBG03,BSBG04,BSBG05A,BSBG05B,...,BSBGSCP,BSDGSCP,BSBGSEC,BSDGSEC,BSDG05S,BSDGEDUP,BSDMLOWP,BSDSLOWP,VERSION,SCOPE
count,8458.0,8458,8458.0,8458.0,8458.0,8456,7314,8341,8373,8366,...,7794.00000,7794,0,0,8235,8370,8431,8444,8458.0,8458
unique,1.0,14,251.0,261.0,8458.0,2,4,5,2,2,...,269.00000,3,0,0,3,6,2,2,1.0,1
top,504.0,Booklet 04,5076.0,524205.0,50010201.0,Girl,Sometimes,None or very few (0–10 books),Yes,Yes,...,9.41328,Somewhat Confident in Physics,NaN,NaN,Either Own Room or Internet Connection,"Some Primary, Lower Secondary or No School",No,No,4.0,Public Use File (PUF)
freq,8458.0,612,75.0,47.0,1.0,4237,3470,4139,5647,4310,...,1167.00000,3412,NaN,NaN,3418,2375,6237,7436,8458.0,8458


## Data understanding:

 ### In the T19_G8_Codebook.xlsx --> 
 #### BSGM7 sheet which contains "Student Context Data Files"
 we find:
 - Columns related to General information about the student: [4:52]
    - Columns related to Student's personal information: [4:7]
    - Columns related to Student's technology belongings: [7:17]
    - Columns related to Parent's education: [17:19]
    - Column about How far does the student think he/she could go in education: [19]
    - Column about How often the student is absent from school: [24]
    - Columns related to Student's Internet usage: [27:33]
    - Columns related to Student's school perception: [33:38]
    - Columns related to Student's feelings: [38:52]
 
 - Columns related to Student's math Likability : [52:62] like ("Enjoy learning math", "Math is boring" ..etc)
 - Columns related to Teachers from students point of view: [62:69]
 - Columns related to Class from student's point of view: [69:75]
 - Columns related to Studen't math perception: [75:84]
 - Columns related to Student's math importance perception [84:93]
 - Columns related to Student's math homeworks & Extra lessons: [128,130,132,134,249,254,259,261]
 - Columns related to unforced difficulties or errors during the plausible_math [263:269] like ("No good place for answer","Slow computer or tablet" ..etc)
 - Columns related to Student's technology usage: [269,288]
 - Columns related to weight: [298,309]
 - Columns related to Plausible value mathematics score: [309:314]
 - Columns related to PV ALGEBRA: [319:324]
 - Columns related to PV MATH APPLYING: [324:329]
 - Columns related to PV DATA & PROBABILITY: [329:334]
 - Columns related to PV GEOMETRY: [334:339]
 - Columns related to PV MATH KNOWING: [339:344]
 - Columns related to PV NUMBER: [344:349]
 - Columns related to PV MATH REASONING: [349:354]
 - Columns related to MATH BENCH REACHED WITH ALL PVs: [394:399]
 - Columns summarizing student's math data: [404:420]
 - Columns related to Derived data: [454:457] 

### in this notebook, We try to replicate the country's score calculation in order to make sure that we are trying to find the right score for students in our model. 
#### We have to understand what are the columns from which the country's math score is generated:

In what concerns our data, we know that Morocco scored : **388** in the **8th** grade, and **383** in the **4th** grade as published by TIMSS in their final report.

### HOW IS THE FINAL SCORE CALCULATED?
While we couldn't find any direct answer to this question, we thoughtfully read many official resources and references. We believe the country score is most likely being calculated as follows:
1. Creating the mean of all "plausible math values" (PV) for each student:
 **PV** = mean(1st_math_PV,2nd_math_PV,3rd_math_PV,4th_math_PV,5th_math_PV)
2. Multiply each **mean_plausible_math_score**  of each student by the Total weight of that student ( in **TOTWGT** column)
3. Calculate the Moroccan Score: 
$$ SCORE_{4^{th}G} = \frac {\Sigma_{i=1}^{n=7723} (PV_{i} \times WEIGHT_{i})}{\Sigma_{i=1}^{n=7723}  WEIGHT_{i}}$$  
with 7723 being the number of Moroccan students participating in the study from 4th grade.

$$ SCORE_{8^{th}G} = \frac {\Sigma_{i=1}^{n=8458} (PV_{i} \times WEIGHT_{i})}{\Sigma_{i=1}^{n=8458}  WEIGHT_{i}}$$ 
with 8458 being the number of Moroccan students participating in the study from 8th grade.

Let's find out:

In [84]:
# We Extract only plausible math values for each grade into a separate dataframe:

df_plausible_math_g8 = df_original_G8.iloc[:, np.r_[309:314]]
df_plausible_math_g4 = df_original_G4.iloc[:, np.r_[132:137]]
#df_plausible_math = df_original_G8.iloc[:, np.r_[4:7,19,24,349:354]]

df_plausible_math_g4.sample(3)

,ASMMAT01,ASMMAT02,ASMMAT03,ASMMAT04,ASMMAT05
5103,538.07140,482.29411,474.37512,501.91816,442.52379
6177,430.23881,405.28422,437.27760,412.67185,402.53143
7260,432.81496,433.65853,443.51013,393.51714,430.52613


In [85]:
df_plausible_math_g8.describe()

,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05
count,8458.00000,8458.00000,8458.00000,8458.00000,8458.00000
unique,8427.00000,8440.00000,8438.00000,8438.00000,8443.00000
top,349.91873,338.13368,358.37952,374.83693,419.64798
freq,2.00000,2.00000,2.00000,2.00000,2.00000


In [86]:
df_plausible_math_g4.describe()

,ASMMAT01,ASMMAT02,ASMMAT03,ASMMAT04,ASMMAT05
count,7723.00000,7723.00000,7723.00000,7723.00000,7723.0000
unique,7712.00000,7713.00000,7714.00000,7713.00000,7713.0000
top,455.54443,343.67081,486.77344,356.35402,436.4955
freq,2.00000,2.00000,2.00000,2.00000,2.0000


In [ ]:
# We have 5 plausible math values columns for each student. All the students in both grades are assigned a value
# in all 5 columns. 
# Let's create a new column that contains the mean plausible math values:

df_plausible_math_g8['mean_math_plausible_score']=df_plausible_math_g8.mean(axis=1)
df_plausible_math_g4['mean_math_plausible_score']=df_plausible_math_g4.mean(axis=1)

In [ ]:
"""
TIMSS assigns crucial weights for ensuring that the sample of students is representative of the entire population,
and they allow for the calculation of accurate population-level statistics. 
The weights are typically calculated in several steps, but TIMSS assigned a total weight (see column "TOTWGT") 
to each student representing all the weights. The weight value is essential to appropriately calculate 
a country's final score.
"""
# Let's add the TOTWGT column to our  "plausible math values" dataframe
df_plausible_math_g8['TOTWGT']=df_original_G8['TOTWGT']
df_plausible_math_g4['TOTWGT']=df_original_G4['TOTWGT']


In [93]:
df_plausible_math_g4.sample(3)

,ASMMAT01,ASMMAT02,ASMMAT03,ASMMAT04,ASMMAT05,mean_math_plausible_score,TOTWGT
4809,478.47476,452.03362,424.13111,479.09494,465.26622,459.800130,231.437679
1283,384.25723,430.46844,375.61210,375.60256,455.65082,404.318230,179.232090
3047,564.79212,460.17486,474.75888,443.25038,489.63808,486.522864,1.000000


In [91]:
df_plausible_math_g8['TOTWGT'].dtype.name

'category'

In [ ]:
# Modify the TOTWGT data type
df_plausible_math_g8['TOTWGT'] = df_plausible_math_g8['TOTWGT'].astype(float)
df_plausible_math_g4['TOTWGT'] = df_plausible_math_g4['TOTWGT'].astype(float)

In [63]:
# Let's sum the TOTAL WEIGHT
Sum_of_weights_g8 = df_plausible_math_g8['TOTWGT'].sum()
Sum_of_weights_g4 = df_plausible_math_g4['TOTWGT'].sum()

674824.316478

In [ ]:
df_plausible_math_g8['student_weighted_score'] = df_plausible_math_g8['mean_math_plausible_score'] * df_plausible_math_g8['TOTWGT']
df_plausible_math_g4['student_weighted_score'] = df_plausible_math_g4['mean_math_plausible_score'] * df_plausible_math_g4['TOTWGT']

In [65]:
sum_of_scores_g8 = df_plausible_math_g8['student_weighted_score'].sum()
sum_of_scores_g4 = df_plausible_math_g4['student_weighted_score'].sum()

258719536.9458718

In [99]:
Moroccan_score_g8 = sum_of_scores_g8 / Sum_of_weights_g8
Moroccan_score_g4 = sum_of_scores_g4 / Sum_of_weights_g4

print(f'Moroccan_score_g8 = {round(Moroccan_score_g8)}  and \nMoroccan_score_g4 = {round(Moroccan_score_g4)}')

Moroccan_score_g8 = 388  and 
Moroccan_score_g4 = 383


**Bingoo!!**

Moroccan_score_g8 = **388**  and Moroccan_score_g4 = **383** are correct. These are the same scores officially published by TIMSS.

Now that we are sure about how the countries' scores are calculated. Let's try to find a model that predicts the mean plausible math score of a particular student using some of the context data.

Before that Let's do some data wrangling.